In [ ]:
import toml
import pygsheets
from tqdm.auto import tqdm
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import sqlalchemy
from sqlalchemy import select, text
import xmltodict
from lxml import etree
from cytoolz import get_in
from datetime import datetime

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.cloning.registry as registry
import paulssonlab.cloning.io as cio
import paulssonlab.cloning.sequence as sequence
import paulssonlab.cloning.primers as primers
import paulssonlab.api.geneious as geneious
import paulssonlab.api.geneious_db as g

# Setup

In [ ]:
config = toml.load("config.toml")

In [ ]:
gc = pygsheets.authorize(service_account_file="credentials.json")

In [ ]:
geneious_sessionmaker = geneious.connect(**config["geneious_test_local"])

In [ ]:
reg = registry.Registry(
    gc,
    config["registry"]["folder"],
    geneious_sessionmaker=geneious_sessionmaker,
    geneious_folder=("registry", "test1"),
)

# Config

In [ ]:
lib_parts = reg[("LIB", "parts")]
olib_oligos = reg[("oLIB", "oligos")]
plib_maps = reg[("pLIB", "maps")]
part_types = reg[("LIB", "parts", "Part types")]

# Sync

In [ ]:
session = reg.geneious_sessionmaker()

In [ ]:
root_folder = session.execute(
    select(g.Folder).where(g.Folder.parent_folder_id == None)
).one()[0]

In [ ]:
geneious_registry_folder = get_in(reg.geneious_folder, root_folder)

In [ ]:
def ensure_folder(path, root):
    folder = root
    for key in path:
        print(key, path)
        if key in folder:
            folder = folder[key]
        else:
            # session.add(g.Folder(name=key, parent_folder=folder, g_group_id=folder.g_group_id))
            new_folder = g.Folder(name=key, g_group_id=folder.g_group_id)
            folder.children[key] = new_folder
            folder = new_folder
    return folder


def update_mtimes(folder):
    while folder is not None:
        folder.modified = text("current_timestamp")
        folder = folder.parent_folder

In [ ]:
test2 = ensure_folder(reg.geneious_folder + ("test2",), root_folder)

In [ ]:
z = session.execute(select(g.AnnotatedDocument)).all()[1][0]

In [ ]:
z

In [ ]:
print(z.document_xml)

In [ ]:
print(z.plugin_document_xml)

In [ ]:
y = session.execute(select(g.AdditionalDocumentXml)).all()  # [3][0]

In [ ]:
y

In [ ]:
print(y[0][0].xml_element)

In [ ]:
print(y.xml_element)

In [ ]:
session.execute(select(g.AnnotatedDocument)).all()

In [ ]:
x = session.execute(select(g.AnnotatedDocument)).all()[0][0]

In [ ]:
session.delete(x)

In [ ]:
session.commit()

In [ ]:
idx = 0
name = f"seq_{idx}"
description = f"seq_descr_{idx}"
# urn = geneious.make_urn()
urn = f"urn:local:jjjj{idx}:k-ehu829p"
plugin_urn = f"urn:sequence:genbank:129361{idx}"
timestamp = geneious.geneious_timestamp()
# timestamp = 1644517862199
dt = datetime.fromtimestamp(timestamp / 1000)

In [ ]:
# s = cio.read_file("/Users/jacob/Downloads/_temp/raw_sample_gb.gb")
s = cio.read_file("/Users/jacob/Downloads/_plasmids/pLIB112.gbk")

In [ ]:
document_xml, plugin_document_xml = geneious.dumps_geneious(
    s,
    urn,
    document_type="genbank",
    name=name,
    description=description,
    timestamp=timestamp,
    pretty_print=True,
)

In [ ]:
adoc = g.AnnotatedDocument(
    parent_folder=test2,
    urn=urn,
    modified=dt,
    document_xml=document_xml,
    plugin_document_xml=plugin_document_xml,
    reference_count=0,
)
# adoc.g_users.append(user)
session.add(adoc)
# session.add(g.AdditionalXmlTimestamp(document_urn=urn, g_user_id=-1, modified=dt))
# session.add(g.AdditionalDocumentXml(document_urn=urn, element_key="sequenceViewOptions", xml_element=sequence_view_options_xml, geneious_major_version_1=2022, geneious_major_version_2=0, g_user_id=-1))
# session.add(g.AdditionalDocumentXml(document_urn=urn, element_key="DocumentHistory", xml_element=document_history_xml, geneious_major_version_1=2022, geneious_major_version_2=0, g_user_id=-1))
# session.add(g.AdditionalDocumentXml(document_urn=urn, element_key="annotations_sequenceViewDedicatedAnnotationsOptions", xml_element=sequence_view_dedicated_xml, geneious_major_version_1=2022, geneious_major_version_2=0, g_user_id=-1))
test2.modified = dt  # text("current_timestamp")

In [ ]:
session.commit()

In [ ]:
user = session.execute(select(g.GUser).where(g.GUser.username == "remoteuser")).one()[0]

In [ ]:
print(document_xml)

In [ ]:
print(plugin_document_xml)

In [ ]:
session.commit()

In [ ]:
special_element_xml = """<NoteType>
  <name>Primer Info</name>
  <code>primerInfo</code>
  <description>Information on a primer</description>
  <isVisible>true</isVisible>
  <isDefaultVisibleInTable>true</isDefaultVisibleInTable>
  <modifiedDate>1633636573500</modifiedDate>
  <fields>
    <field>
      <name>Gene</name>
      <description />
      <code>gene</code>
      <editable>true</editable>
      <type>java.lang.String</type>
      <constraints />
    </field>
    <field enumeratedType="true">
      <name>Direction</name>
      <description>Direction of the primer</description>
      <code>direction</code>
      <editable>true</editable>
      <type>java.lang.String</type>
      <valueSet>
        <value>Forward</value>
        <value>Reverse</value>
      </valueSet>
      <constraints />
    </field>
    <field>
      <name>Box</name>
      <description />
      <code>box</code>
      <editable>true</editable>
      <type>java.lang.String</type>
      <constraints />
    </field>
    <field>
      <name>Row</name>
      <description />
      <code>row</code>
      <editable>true</editable>
      <type>java.lang.String</type>
      <constraints />
    </field>
    <field>
      <name>Column</name>
      <description />
      <code>column</code>
      <editable>true</editable>
      <type>java.lang.String</type>
      <constraints />
    </field>
    <field>
      <name>Purification Method</name>
      <description />
      <code>purificationMethod</code>
      <editable>true</editable>
      <type>java.lang.String</type>
      <constraints />
    </field>
    <field>
      <name>PMOL</name>
      <description />
      <code>pmol</code>
      <editable>true</editable>
      <type>java.lang.String</type>
      <constraints />
    </field>
    <field>
      <name>In Stock</name>
      <description />
      <code>inStock</code>
      <editable>true</editable>
      <type>java.lang.Boolean</type>
      <constraints />
    </field>
    <field>
      <name>Notes</name>
      <description />
      <code>notes</code>
      <editable>true</editable>
      <type>java.lang.String</type>
      <constraints />
    </field>
  </fields>
</NoteType>"""

In [ ]:
sequence_view_options_xml = """<sequenceViewOptions geneiousVersion="2022.0.2">
  <SequenceViewerState>
    <zoom isMinimumZoom="true">1.0</zoom>
    <SequenceViewerSelectionState>
      <selectedRegions>
        <element class="com.biomatters.plugins.sequenceviewer.selection.SelectedRegion">
          <SelectedRegion>
            <residues>
              <XMLSerialisableRootElement>
                <from>28</from>
                <to>28</to>
              </XMLSerialisableRootElement>
            </residues>
            <sequences>
              <XMLSerialisableRootElement>
                <from>0</from>
                <to>1</to>
              </XMLSerialisableRootElement>
            </sequences>
          </SelectedRegion>
        </element>
      </selectedRegions>
    </SequenceViewerSelectionState>
    <SequenceViewerLocation>
      <sequenceNumber>0</sequenceNumber>
      <residueNumber>22</residueNumber>
      <annotationNumber>0</annotationNumber>
      <annotation_Track>-1</annotation_Track>
      <annotationIntervalNumber>0</annotationIntervalNumber>
      <xOffset>-1</xOffset>
      <yOffset>-23</yOffset>
    </SequenceViewerLocation>
    <docRevisionNumbers>
      <listValue>1</listValue>
    </docRevisionNumbers>
  </SequenceViewerState>
  <translation>
    <option name="frame">Frame1</option>
    <option name="relativeTo">sequence</option>
  </translation>
  <liveAnnotators>
    <option name="com.biomatters.plugins.featureFinder.FeatureFinder_NUCLEOTIDEOn">false</option>
    <option name="ORF_Annotation_GeneratorOn">false</option>
  </liveAnnotators>
  <restrictionSites>
    <childOption name="com.biomatters.plugins.restrictionAnalysis.RestrictionSiteAnnotationGenerator">
      <childOption name="com.biomatters.plugins.restrictionAnalysis.RestrictionSiteAnnotationGenerator">
        <option name="excluded">(1, 28)</option>
        <option name="grabExcludeRangeOption">[1,28)</option>
      </childOption>
    </childOption>
  </restrictionSites>
</sequenceViewOptions>"""

document_history_xml = """<DocumentHistory geneiousVersion="2022.0.2">
  <entryFields>
    <DocumentHistoryEntryField class="com.biomatters.geneious.publicapi.documents.DocumentHistoryEntryFieldImplementation">
      <com.biomatters.date fieldName="Created">1626900476656</com.biomatters.date>
    </DocumentHistoryEntryField>
    <DocumentHistoryEntryField class="com.biomatters.geneious.publicapi.documents.DocumentHistoryEntryFieldImplementation">
      <com.biomatters.user fieldName="User">jacob</com.biomatters.user>
    </DocumentHistoryEntryField>
    <DocumentHistoryEntryField class="com.biomatters.geneious.publicapi.documents.DocumentHistoryEntryFieldImplementation">
      <com.biomatters.geneiousVersion fieldName="Geneious Version">2021.2.2</com.biomatters.geneiousVersion>
    </DocumentHistoryEntryField>
    <DocumentHistoryEntryField class="com.biomatters.geneious.publicapi.documents.DocumentHistoryEntryFieldImplementation">
      <com.biomatters.autogeneratedEntry fieldName="" />
    </DocumentHistoryEntryField>
  </entryFields>
</DocumentHistory>"""

sequence_view_dedicated_xml = """<annotations_sequenceViewDedicatedAnnotationsOptions geneiousVersion="2022.0.2">
  <SequenceViewerState>
    <zoom isMinimumZoom="true">0.0</zoom>
    <SequenceViewerSelectionState tmForSelection="-1.0">
      <selectedRegions>
        <element class="com.biomatters.plugins.sequenceviewer.selection.SelectedRegion">
          <SelectedRegion>
            <residues>
              <XMLSerialisableRootElement>
                <from>0</from>
                <to>28</to>
              </XMLSerialisableRootElement>
            </residues>
            <sequences>
              <XMLSerialisableRootElement>
                <from>0</from>
                <to>1</to>
              </XMLSerialisableRootElement>
            </sequences>
          </SelectedRegion>
        </element>
      </selectedRegions>
    </SequenceViewerSelectionState>
  </SequenceViewerState>
  <translation>
    <option name="frame">Frame1</option>
    <option name="relativeTo">sequence</option>
  </translation>
</annotations_sequenceViewDedicatedAnnotationsOptions>"""

In [ ]:
session.execute(select(g.SpecialElement).where(g.SpecialElement))

In [ ]:
annotation_definitions_folder = session.execute(
    select(g.Folder)
    .where(g.Folder.name == "annotationDefinitions")
    .execution_options(include_invisible=True)
).one()[0]
# session.execute(update(g.SpecialElement))

In [ ]:
annotation_definitions_folder  # .special_elements

In [ ]:
print(x.document_xml)

In [ ]:
adoc

In [ ]:
# parent folder modified
# update special_element for invisible annotationDefinitions folder
# parent folder modified
# insert into document_read [DONE]
# insert into indexing_queue
# additional_document_xml [DONE]
# try to update additional_xml_timestamp
# if fails, insert additional_xml_timestamp [DONE]

In [ ]:
session.execute(
    select(g.Folder).where(g.Folder.id == 462).execution_options(include_invisible=True)
).all()

In [ ]:
session.commit()

In [ ]:
# execute <unnamed>: UPDATE special_element SET modified = CURRENT_TIMESTAMP, xml = $1 WHERE name = $2 AND folder_id = $3

"""
parameters: $1 = '<NoteType>
  <name>Primer Info</name>
  <code>primerInfo</code>
  <description>Information on a primer</description>
  <isVisible>true</isVisible>
  <isDefaultVisibleInTable>true</isDefaultVisibleInTable>
  <modifiedDate>1633636573500</modifiedDate>
  <fields>
    <field>
      <name>Gene</name>
      <description />
      <code>gene</code>
      <editable>true</editable>
      <type>java.lang.String</type>
      <constraints />
    </field>
    <field enumeratedType="true">
      <name>Direction</name>
      <description>Direction of the primer</description>
      <code>direction</code>
      <editable>true</editable>
      <type>java.lang.String</type>
      <valueSet>
        <value>Forward</value>
        <value>Reverse</value>
      </valueSet>
      <constraints />
    </field>
    <field>
      <name>Box</name>
      <description />
      <code>box</code>
      <editable>true</editable>
      <type>java.lang.String</type>
      <constraints />
    </field>
    <field>
      <name>Row</name>
      <description />
      <code>row</code>
      <editable>true</editable>
      <type>java.lang.String</type>
      <constraints />
    </field>
    <field>
      <name>Column</name>
      <description />
      <code>column</code>
      <editable>true</editable>
      <type>java.lang.String</type>
      <constraints />
    </field>
    <field>
      <name>Purification Method</name>
      <description />
      <code>purificationMethod</code>
      <editable>true</editable>
      <type>java.lang.String</type>
      <constraints />
    </field>
    <field>
      <name>PMOL</name>
      <description />
      <code>pmol</code>
      <editable>true</editable>
      <type>java.lang.String</type>
      <constraints />
    </field>
    <field>
      <name>In Stock</name>
      <description />
      <code>inStock</code>
      <editable>true</editable>
      <type>java.lang.Boolean</type>
      <constraints />
    </field>
    <field>
      <name>Notes</name>
      <description />
      <code>notes</code>
      <editable>true</editable>
      <type>java.lang.String</type>
      <constraints />
    </field>
  </fields>
</NoteType>', $2 = '2', $3 = '2'
"""

In [ ]:
# execute S_8: INSERT INTO additional_document_xml(xml_element, document_urn, element_key, g_user_id, geneious_major_version_1, geneious_major_version_2) VALUES($1, $2, $3, $4, $5, $6)

"""
parameters: $1 = '<sequenceViewOptions geneiousVersion="2022.0.2">
  <SequenceViewerState>
    <zoom isMinimumZoom="true">1.0</zoom>
    <SequenceViewerSelectionState>
      <selectedRegions>
        <element class="com.biomatters.plugins.sequenceviewer.selection.SelectedRegion">
          <SelectedRegion>
            <residues>
              <XMLSerialisableRootElement>
                <from>28</from>
                <to>28</to>
              </XMLSerialisableRootElement>
            </residues>
            <sequences>
              <XMLSerialisableRootElement>
                <from>0</from>
                <to>1</to>
              </XMLSerialisableRootElement>
            </sequences>
          </SelectedRegion>
        </element>
      </selectedRegions>
    </SequenceViewerSelectionState>
    <SequenceViewerLocation>
      <sequenceNumber>0</sequenceNumber>
      <residueNumber>22</residueNumber>
      <annotationNumber>0</annotationNumber>
      <annotation_Track>-1</annotation_Track>
      <annotationIntervalNumber>0</annotationIntervalNumber>
      <xOffset>-1</xOffset>
      <yOffset>-23</yOffset>
    </SequenceViewerLocation>
    <docRevisionNumbers>
      <listValue>1</listValue>
    </docRevisionNumbers>
  </SequenceViewerState>
  <translation>
    <option name="frame">Frame1</option>
    <option name="relativeTo">sequence</option>
  </translation>
  <liveAnnotators>
    <option name="com.biomatters.plugins.featureFinder.FeatureFinder_NUCLEOTIDEOn">false</option>
    <option name="ORF_Annotation_GeneratorOn">false</option>
  </liveAnnotators>
  <restrictionSites>
    <childOption name="com.biomatters.plugins.restrictionAnalysis.RestrictionSiteAnnotationGenerator">
      <childOption name="com.biomatters.plugins.restrictionAnalysis.RestrictionSiteAnnotationGenerator">
        <option name="excluded">(1, 28)</option>
        <option name="grabExcludeRangeOption">[1,28)</option>
      </childOption>
    </childOption>
  </restrictionSites>
</sequenceViewOptions>', $2 = 'urn:local:jacob:k-ei0r6mv', $3 = 'sequenceViewOptions', $4 = '-1', $5 = '2022', $6 = '0'
"""

In [ ]:
"""
parameters: $1 = '<DocumentHistory geneiousVersion="2022.0.2">
  <entryFields>
    <DocumentHistoryEntryField class="com.biomatters.geneious.publicapi.documents.DocumentHistoryEntryFieldImplementation">
      <com.biomatters.date fieldName="Created">1626900476656</com.biomatters.date>
    </DocumentHistoryEntryField>
    <DocumentHistoryEntryField class="com.biomatters.geneious.publicapi.documents.DocumentHistoryEntryFieldImplementation">
      <com.biomatters.user fieldName="User">jacob</com.biomatters.user>
    </DocumentHistoryEntryField>
    <DocumentHistoryEntryField class="com.biomatters.geneious.publicapi.documents.DocumentHistoryEntryFieldImplementation">
      <com.biomatters.geneiousVersion fieldName="Geneious Version">2021.2.2</com.biomatters.geneiousVersion>
    </DocumentHistoryEntryField>
    <DocumentHistoryEntryField class="com.biomatters.geneious.publicapi.documents.DocumentHistoryEntryFieldImplementation">
      <com.biomatters.autogeneratedEntry fieldName="" />
    </DocumentHistoryEntryField>
  </entryFields>
</DocumentHistory>', $2 = 'urn:local:jacob:k-ei0r6mv', $3 = 'DocumentHistory', $4 = '-1', $5 = '2022', $6 = '0'
"""

In [ ]:
# can skip??

"""
parameters: $1 = '<annotations_sequenceViewDedicatedAnnotationsOptions geneiousVersion="2022.0.2">
  <SequenceViewerState>
    <zoom isMinimumZoom="true">0.0</zoom>
    <SequenceViewerSelectionState tmForSelection="-1.0">
      <selectedRegions>
        <element class="com.biomatters.plugins.sequenceviewer.selection.SelectedRegion">
          <SelectedRegion>
            <residues>
              <XMLSerialisableRootElement>
                <from>0</from>
                <to>28</to>
              </XMLSerialisableRootElement>
            </residues>
            <sequences>
              <XMLSerialisableRootElement>
                <from>0</from>
                <to>1</to>
              </XMLSerialisableRootElement>
            </sequences>
          </SelectedRegion>
        </element>
      </selectedRegions>
    </SequenceViewerSelectionState>
  </SequenceViewerState>
  <translation>
    <option name="frame">Frame1</option>
    <option name="relativeTo">sequence</option>
  </translation>
</annotations_sequenceViewDedicatedAnnotationsOptions>', $2 = 'urn:local:jacob:k-ei0r6mv', $3 = 'annotations_sequenceViewDedicatedAnnotationsOptions', $4 = '-1', $5 = '2022', $6 = '0'
"""

In [ ]:
for key in reg:
    type_ = key[1]
    if key[0] == "LIB" and key[1] in ("oligos",):
        client = reg[key]
        collection_folder = ensure_folder(key, geneious_registry_folder)
        for name in client.keys():
            entry = reg.get_with_prefix(name, key[0], type_)
            print(name)
            print(entry)
            if type_ == "parts":
                pass
            elif type_ == "plasmids":
                pass
            elif type_ == "oligos":
                urn = geneious.make_urn()

            else:
                raise NotImplementedError
            break

In [ ]:
geneious.make_urn()

In [ ]:
a, b = geneious.dump_geneious(
    s,
    geneious.make_urn(),
    document_type="primer",
    name="myname",
    description="mydescr",
    timestamp=geneious.geneious_timestamp(),
)

In [ ]:
print(etree.tostring(b, encoding="utf8", pretty_print=True).decode())

In [ ]:
dir(s)

In [ ]:
print(
    get_in(("New Folder", "oligos"), root_folder)
    .annotated_documents[0]
    .plugin_document_xml
)

In [ ]:
x = session.execute(select(g.AnnotatedDocument)).all()[1][0]

In [ ]:
d1 = x.plugin_document_xml

In [ ]:
print(d1)

# Test

## DB

In [ ]:
db_url = sqlalchemy.engine.URL.create(**config["geneious_test_local"])
engine = sqlalchemy.create_engine(db_url, future=True, echo=True)
Session = sqlalchemy.orm.sessionmaker(bind=engine, future=True)

## DB test

In [ ]:
session = Session()

In [ ]:
x = session.execute(select(g.Folder).where(g.Folder.name == "plasmids")).one()[0]

In [ ]:
x.annotated_documents

In [ ]:
y = x.annotated_documents[0]

In [ ]:
y.additional_document_xml

In [ ]:
session.execute("SELECT max(id) FROM folder").one()

In [ ]:
session.execute("SELECT current_timestamp()").one()

In [ ]:
session.execute("SELECT document_urn, element_key FROM additional_document_xml").all()

In [ ]:
import datetime

In [ ]:
datetime.datetime.now()

In [ ]:
# start another geneious client (with another postgres user), add folder, see what triggers select from next_table_id

# check cadence of select, see if the big join produces expected output
# (difference between python folders and geneious folders?)

# try adding new folder, changing modified of parent folder, see if geneious selects from next_table_id

In [ ]:
# 2022-02-21 17:26:39.479 EST,"newuser","geneious_test1",23897,"127.0.0.1:64955",62140436.5d59,1580,"SELECT",2022-02-21 16:29:26 EST,4/1573,0,LOG,00000,"execute S_4: SELECT A.id,A.name,g_group_id,A.modified,document_urn,parent_folder_id,visible, C.xml AS folderColourXml, (CASE WHEN C.modified IS NOT NULL THEN C.modified ELSE CURRENT_TIMESTAMP END) AS folderColourModified FROM folder A LEFT OUTER JOIN folder_view B ON A.id = B.folder_id LEFT OUTER JOIN special_element C ON A.id = C.folder_id AND C.name = 'folderColor' WHERE A.modified > $1 AND A.modified < $2 AND id NOT IN (select hidden_folder_id from hidden_folder_to_user WHERE user_id != $3)","parameters: $1 = '2022-02-21 17:00:43.453', $2 = '2022-02-21 17:26:39.479', $3 = '1'",,,,,,,,"PostgreSQL JDBC Driver","client backend",,0

In [ ]:
for idx in range(40):
    with Session.begin() as session:
        group = session.execute(
            select(g.GGroup).where(g.GGroup.name == "Everybody")
        ).one()[0]
        print("group", group)
        parent = session.execute(select(g.Folder).where(g.Folder.name == "xxx5")).one()[
            0
        ]
        print(parent)
        new_name = f"sub{idx}"
        new_folder = g.Folder(name=new_name, g_group=group)
        new_folder.parent_folder = parent
        parent.modified = text("current_timestamp")  # datetime.datetime.utcnow()
        session.add(new_folder)
        # parent.children[new_name] = new_folder
        # session.execute(update(sql.functions.max(g.Table.id)))
        # session.execute("UPDATE next_table_id SET next_id = (SELECT max(id) FROM folder) WHERE table_name = 'folder'")

In [ ]:
with Session.begin() as session:
    q = select(
        g.Folder
    )  # .where(g.Folder.name == "bot4")#.where(g.Folder.visible == True)
    res = session.execute(q).all()
    print("\n".join(map(str, res)))

In [ ]:
with Session.begin() as session:
    q = select(g.NextTableId)
    res = session.execute(q).all()
    print("\n".join(map(str, res)))

In [ ]:
with Session.begin() as session:
    res = session.execute(text("SELECT * from folder;")).all()
    print("\n".join(map(str, res)))

In [ ]:
with Session.begin() as session:
    q = select(g.NextTableId)
    res = session.execute(q).all()
    print("\n".join(map(str, res)))

In [ ]:
with Session.begin() as session:
    q = select(g.Folder)
    res = session.execute(q).all()
    print("\n".join(map(str, res)))

## XML

In [ ]:
seq = plib_maps["pLIB112"]

In [ ]:
from xmldiff.main import diff_texts

In [ ]:
output = etree.tostring(geneious.dump_geneious(seq4), pretty_print=True)

In [ ]:
x = etree.fromstring(doc4)

In [ ]:
geneious.load_geneious(x)

In [ ]:
seq4.annotations

In [ ]:
seq4.annotations["references"][0].__class__

In [ ]:
geneious.loads_geneious(doc4).annotations

## Round-trip: seq->geneious-seq

## Round-trip: geneious->seq->geneious

## Old

In [ ]:
for name, t in metadata.tables.items():
    print(name)
    q = select([t]).limit(20)
    rows = conn.execute(q).all()
    for row in rows:
        print(row)
    print()

In [ ]:
q = select(g.AnnotatedDocument)
rows = conn.execute(q).all()

In [ ]:
doc0 = rows[0]["plugin_document_xml"]

In [ ]:
doc4 = rows[-1]["plugin_document_xml"]

In [ ]:
import os

In [ ]:
seq4 = cio.read_file(os.path.expanduser("/Users/jacob/Downloads/_plasmids/genbank2.gb"))

In [ ]:
seq4

In [ ]:
print(doc4)

In [ ]:
doc2 = rows[1]["plugin_document_xml"]

In [ ]:
rows[5]["xml_element"]

In [ ]:
docs = [xmltodict.parse(row["document_xml"], dict_constructor=dict) for row in rows]

In [ ]:
BAD_CLASSES = [
    # "com.biomatters.geneious.publicapi.implementations.sequence.OligoSequenceDocument",
    "com.biomatters.geneious.publicapi.documents.OperationRecordDocument",
]
good_docs = [d for d in docs if d["document"]["@class"] not in BAD_CLASSES]

In [ ]:
set(d["document"]["@class"] for d in good_docs)

In [ ]:
rows[2]["document_xml"]

In [ ]:
x = xmltodict.parse(rows[2]["plugin_document_xml"], dict_constructor=dict)

In [ ]:
x